In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q transformers accelerate sentencepiece torch


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import json
import re
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM

# -----------------------------
# CONFIG
# -----------------------------
MODEL_NAME = "google/medgemma-4b-it" 
device = "cuda" if torch.cuda.is_available() else "cpu"

def load_model():
    print(f"Loading model {MODEL_NAME} on {device}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_model()

In [7]:
import torch
import json
import re
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM

# -----------------------------
# CONFIG
# -----------------------------
MODEL_NAME = "google/medgemma-4b-it" 
device = "cuda" if torch.cuda.is_available() else "cpu"

def load_model():
    print(f"Loading model {MODEL_NAME} on {device}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_model()

Loading model google/medgemma-4b-it on cuda...


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

In [5]:


# -----------------------------
# QUESTION DATA STRUCTURE
# -----------------------------
QUESTIONS = {
    "General": {
        "Q1": {
            "en": "What is the child’s age?", 
            "ml": "കുട്ടിയുടെ വയസ് എത്രയാണ്?", 
            "type": "number", "min": 6, "max": 12,
            "context": lambda v: f"The child is {v} years old."
        },
        "Q2": {
            "en": "How long has the problem been present?", 
            "ml": "ഈ പ്രശ്നം എത്ര ദിവസമായി തുടരുന്നു?", 
            "type": "radio", 
            "options": {
                "< 1 day": {"en": "< 1 day", "ml": "1 ദിവസത്തിൽ താഴെ"},
                "1–2 days": {"en": "1–2 days", "ml": "1-2 ദിവസം"},
                "3+ days": {"en": "3+ days", "ml": "3 ദിവസത്തിലധികം"}
            },
            "context": {
                "< 1 day": "The symptoms started recently (less than 24 hours ago).",
                "1–2 days": "The symptoms have been present for 1 to 2 days.",
                "3+ days": "The symptoms have persisted for more than 3 days."
            }
        },
        "Q3": {
            "en": "Is the child unusually drowsy, confused, or not responding normally?", 
            "ml": "കുട്ടി അസാധാരണമായി ഉറക്കമുള്ളതോ പ്രതികരിക്കാത്തതോ ആണോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "The child is showing signs of altered consciousness, unusual drowsiness, or confusion.",
                "No": "The child is alert and responding normally to surroundings."
            }
        },
        "Q4": {
            "en": "Is the child able to drink and keep fluids down?", 
            "ml": "കുട്ടിക്ക് വെള്ളം കുടിക്കാനും നിലനിർത്താനും കഴിയുന്നുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child is able to tolerate oral fluids and maintain hydration.",
                "No": "The child is unable to drink or keep any fluids down, risking dehydration."
            }
        },
        "Q5": {
            "en": "Does the child have asthma, diabetes, heart disease, or other chronic illness?", 
            "ml": "കുട്ടിക്ക് ആസ്ത്മ, പ്രമേഹം, ഹൃദ്രോഗം തുടങ്ങിയ ദീർഘകാല രോഗങ്ങളുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child has a pre-existing chronic medical condition (e.g., asthma, diabetes).",
                "No": "The child has no known chronic underlying illnesses."
            }
        },
    },
    "Cold / Cough / Fever": {

        "Q6": {
            "en": "How does the fever feel?", 
            "ml": "പനി എങ്ങനെ തോന്നുന്നു?", 
            "type": "radio", 
            "options": {
                "Warm but child active": {"en": "Warm but child active", "ml": "ചെറിയ പനി, കുട്ടി ഉന്മേഷവാനാണ്"},
                "Hot and uncomfortable": {"en": "Hot and uncomfortable", "ml": "ശരീരം നന്നായി ചൂടുണ്ട്, ആസ്വസ്ഥതയുണ്ട്"},
                "Very hot and child weak": {"en": "Very hot and child weak", "ml": "കഠിനമായ പനി, കുട്ടി വളരെ അവശനാണ്"}
            },
            "context": {
                "Warm but child active": "Mild fever with normal activity.",
                "Hot and uncomfortable": "Moderate fever.",
                "Very hot and child weak": "High fever affecting the child."
            }
        },
        "Q7": {
            "en": "Has the fever lasted more than 3 days?", 
            "ml": "പനി 3 ദിവസത്തിലധികമായി തുടരുന്നുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The fever has been prolonged, lasting more than 72 hours.",
                "No": "The fever is recent and has lasted less than 3 days."
            }
        },
        "Q8": {
            "en": "Is there a rash on the body?", 
            "ml": "ശരീരത്തിൽ ചൊറിച്ചിലോ ചർമ്മത്തിൽ പാടുകളോ ഉണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "A new skin rash or spots have appeared on the child's body.",
                "No": "There is no visible rash on the skin."
            }
        },
        "Q9": {
            "en": "Is the child unable to bend the neck forward?", 
            "ml": "കുട്ടിക്ക് കഴുത്ത് മുന്നോട്ട് കുനിക്കാനാകുന്നില്ലേ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "The child is experiencing neck stiffness (meningismus), unable to touch chin to chest.",
                "No": "The child has normal neck mobility."
            }
        },
    },
    "Stomach Pain / Diarrhea / Vomiting": {
        "Q10": {
            "en": "How many times has the child vomited in the last 24 hours?", 
            "ml": "കഴിഞ്ഞ 24 മണിക്കൂറിൽ എത്ര തവണ ഛർദ്ദിച്ചു?", 
            "type": "radio", 
            "options": {
                "None": {"en": "None", "ml": "ഒന്നുമില്ല"},
                "1-3": {"en": "1-3", "ml": "1-3 തവണ"},
                "4+": {"en": "4+", "ml": "4 തവണയിൽ കൂടുതൽ"}
            },
            "context": {
                "None": "The child has not vomited in the last 24 hours.",
                "1-3": "The child has vomited 1 to 3 times recently.",
                "4+": "The child is experiencing frequent/excessive vomiting (4 or more times)."
            }
        },
        "Q11": {
            "en": "Is there blood in vomit or stool?", 
            "ml": "ഛർദ്ദിയിലോ മലത്തിലോ രക്തം ഉണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "Visible blood is present in the child's vomit or bowel movements.",
                "No": "There is no blood observed in vomit or stool."
            }
        },
        "Q12": {
            "en": "Is the stomach pain severe and constant?", 
            "ml": "വയറുവേദന കഠിനവും സ്ഥിരവുമാണോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child is reporting intense, continuous abdominal pain.",
                "No": "Abdominal pain is absent or only mild/intermittent."
            }
        },
        "Q13": {
            "en": "Has the child passed urine in the last 8 hours?", 
            "ml": "കഴിഞ്ഞ 8 മണിക്കൂറിൽ കുട്ടി മൂത്രമൊഴിച്ചിട്ടുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child has normal urine output.",
                "No": "The child has not urinated for over 8 hours, indicating potential dehydration."
            }
        },
    },
    "Breathing Problem": {
        "Q14": {
            "en": "Is the child breathing faster than usual?", 
            "ml": "കുട്ടി സാധാരണയേക്കാൾ വേഗത്തിൽ ശ്വസിക്കുന്നുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child is showing tachypnea (increased breathing rate).",
                "No": "The child's breathing rate is within the normal range."
            }
        },
        "Q15": {
            "en": "Is the chest pulling in while breathing?", 
            "ml": "ശ്വസിക്കുമ്പോൾ നെഞ്ച് ഉള്ളിലേക്ക് വലിക്കപ്പെടുന്നുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "The child has chest retractions (respiratory distress), where the skin pulls in around the ribs.",
                "No": "The child is breathing easily without visible retractions."
            }
        },
        "Q16": {
            "en": "Are the lips or face turning bluish?", 
            "ml": "ചുണ്ടുകളോ മുഖമോ നീല നിറത്തിലാകുന്നുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "Cyanosis is present: The child's lips or face have a blue tint, indicating low oxygen.",
                "No": "The child has normal skin/lip coloration."
            }
        },
        "Q17": {
            "en": "Is the child unable to speak full sentences due to breathlessness?", 
            "ml": "ശ്വാസം മുട്ടലാൽ കുട്ടിക്ക് പൂർണ്ണ വാചകം പറയാനാകുന്നില്ലേ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child is showing severe breathlessness, unable to speak in full sentences.",
                "No": "The child can speak normally without significant shortness of breath."
            }
        },
    },
    "Body Pain / Headache": {
        "Q18": {
            "en": "Is the headache or body pain severe ?", 
            "ml": "തലവേദനയോ ശരീരവേദനയോ കൂടുതലാണോ ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child is in severe headache or body pain.",
                "No": "The child is experiencing mild to moderate pain."
            }
        },
        "Q19": {
            "en": "Did the child have a head injury recently?", 
            "ml": "കുട്ടിക്ക് അടുത്തിടെ തലക്ക് പരിക്കുണ്ടായിട്ടുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "There is a history of recent trauma or injury to the head.",
                "No": "There has been no recent head injury."
            }
        },
        "Q20": {
            "en": "Is there repeated vomiting with headache?", 
            "ml": "തലവേദനയോടൊപ്പം ആവർത്തിച്ച ഛർദ്ദിയുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "context": {
                "Yes": "The child has a headache accompanied by persistent vomiting.",
                "No": "The headache is not associated with vomiting."
            }
        },
    },
    "Critical Red-Flags": {
        "Q21": {
            "en": "Has the child had a seizure?", 
            "ml": "കുട്ടിക്ക് അപസ്മാരം ഉണ്ടായിട്ടുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "The child has experienced a seizure or convulsion.",
                "No": "The child has had no seizures."
            }
        },
        "Q22": {
            "en": "Has the child fainted or become unconscious?", 
            "ml": "കുട്ടി ബോധരഹിതനായിട്ടുണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "The child has experienced loss of consciousness or fainting.",
                "No": "The child has remained conscious throughout."
            }
        },
        "Q23": {
            "en": "Is there a severe injury or heavy bleeding?", 
            "ml": "ഗുരുതരമായ പരിക്കോ രക്തസ്രാവമോ ഉണ്ടോ?", 
            "type": "radio", 
            "options": {"Yes": {"en": "Yes", "ml": "അതെ"}, "No": {"en": "No", "ml": "അല്ല"}}, 
            "is_critical": True,
            "context": {
                "Yes": "The child has sustained a major injury or is actively bleeding heavily.",
                "No": "There is no severe injury or heavy bleeding noted."
            }
        },
    }
}

# -----------------------------
# HOME CARE ADVICE LIBRARY
# -----------------------------
HOME_ADVICE_LIBRARY = {
    "REST": {
        "en": "Ensure the child gets adequate rest.",
        "ml": "കുട്ടിക്ക് ആവശ്യത്തിന് വിശ്രമം നൽകുക."
    },
    "FLUIDS": {
        "en": "Encourage frequent intake of clean fluids like water or coconut water.",
        "ml": "വെള്ളം, ഇളനീർ തുടങ്ങിയ പാനീയങ്ങൾ ധാരാളം നൽകുക."
    },
    "LIGHT_DIET": {
        "en": "Provide light, easily digestible food.",
        "ml": "ലഘുവായതും എളുപ്പത്തിൽ ദഹിക്കുന്നതുമായ ഭക്ഷണം നൽകുക."
    },
    "HYGIENE": {
        "en": "Maintain proper hand hygiene to prevent spread of infection.",
        "ml": "അണുബാധ പടരാതിരിക്കാൻ കൈകൾ വൃത്തിയായി സൂക്ഷിക്കുക."
    },
    "MONITOR_SYMPTOMS": {
        "en": "Monitor symptoms closely for any worsening.",
        "ml": "ലക്ഷണങ്ങൾ കൂടുന്നുണ്ടോ എന്ന് ശ്രദ്ധാപൂർവ്വം നിരീക്ഷിക്കുക."
    },
    "TEMPERATURE_CHECK": {
        "en": "Check temperature periodically if fever is present.",
        "ml": "പനി ഉണ്ടെങ്കിൽ കൃത്യസമയത്ത് താപനില പരിശോധിക്കുക."
    }
}

# -----------------------------
# HARD RED FLAG CHECK
# -----------------------------
def check_red_flags(answers):
    critical_data = {
        "triage_level": "RED",
        "reasoning": "Immediate medical attention required for life-threatening symptoms flagged by critical clinical rules.",
        "confidence": "High (Rule-based Override)",
        "home_advice": []
    }
    
    # Rule 1: Direct Critical Questions (Q21, Q22, Q23)
    if any(answers.get(q) == "Yes" for q in ["Q21", "Q22", "Q23"]):
        return critical_data
    # Rule 2: Q3 (Not responding)
    if answers.get("Q3") == "Yes":
        return critical_data
    # Rule 3: Q9 (Neck Stiffness standalone)
    if answers.get("Q9") == "Yes":
        return critical_data
    # Rule 4: Q11 (Blood in vomit/stool)
    if answers.get("Q11") == "Yes":
        return critical_data
    # Rule 5: Q15/Q16 (Chest pulling / Bluish)
    if answers.get("Q15") == "Yes" or answers.get("Q16") == "Yes":
        return critical_data
    # Rule 6: Head injury + Vomiting (Q19 + Q20)
    if answers.get("Q19") == "Yes" and (answers.get("Q20") == "Yes" or answers.get("Q10", "") == "4+"):
        return critical_data
    
    # Rule 7: Chronic illness + Breathing distress (Q5 + Q14)
    if answers.get("Q5") == "Yes" and answers.get("Q14") == "Yes":
        return critical_data
    
    # Rule 8: Dehydration Risk (No urine + Vomiting) (Q13 + Q10)
    if answers.get("Q13") == "No" and answers.get("Q10", "None") != "None":
        return critical_data
        
    # Rule 9: Severe Pain + Fever Combination (Q18 + Q6)
    if answers.get("Q18") == "Yes" and answers.get("Q6") is not None:
        return critical_data
        
    return None

# -----------------------------
# BUILD STRUCTURED SUMMARY 
# -----------------------------
def build_summary(answers):
    summary = "Pediatric Clinical Assessment (Age 6-12):\n"
    for cat, qs in QUESTIONS.items():
        cat_summary = ""
        for q_id, q_data in qs.items():
            val = answers.get(q_id)
            if val is not None:
                # Use context mapping if available
                if "context" in q_data:
                    if callable(q_data["context"]):
                        desc = q_data["context"](val)
                    elif isinstance(q_data["context"], dict):
                        desc = q_data["context"].get(val, f"{q_data['en']}: {val}")
                    else:
                        desc = f"{q_data['en']}: {val}"
                else:
                    desc = f"{q_data['en']}: {val}"
                cat_summary += f"- {desc}\n"
        
        if cat_summary:
            summary += f"\n### {cat}\n{cat_summary}"
    return summary

# -----------------------------
# MEDGEMMA CLASSIFICATION & EXTRACTION
# -----------------------------
REQUIRED_KEYS = {"triage_level", "reasoning", "confidence", "home_advice"}
VALID_ADVICE = {
    "REST",
    "FLUIDS",
    "LIGHT_DIET",
    "HYGIENE",
    "MONITOR_SYMPTOMS",
    "TEMPERATURE_CHECK"
}
VALID_TRIAGE = {"RED", "YELLOW", "GREEN"}

def extract_json_response(response_text: str):
    """
    Safely extract first valid JSON object from model output.
    """
    # 1️⃣ Remove markdown wrappers if present
    cleaned = response_text.strip()
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()

    # 2️⃣ Find first balanced JSON block
    start = cleaned.find("{")
    if start == -1:
        raise ValueError("No JSON object found")

    brace_count = 0
    for i in range(start, len(cleaned)):
        if cleaned[i] == "{":
            brace_count += 1
        elif cleaned[i] == "}":
            brace_count -= 1

        if brace_count == 0:
            json_str = cleaned[start:i+1]
            break
    else:
        raise ValueError("Incomplete JSON object")

    # 3️⃣ Parse JSON safely
    data = json.loads(json_str)

    # 4️⃣ Validate required fields
    if not REQUIRED_KEYS.issubset(data.keys()):
        raise ValueError("Missing required keys in JSON response")

    # 5️⃣ Validate triage level
    if data.get("triage_level") not in VALID_TRIAGE:
        raise ValueError(f"Invalid triage level: {data.get('triage_level')}")

    # 6️⃣ Validate advice keys
    if not isinstance(data["home_advice"], list):
        raise ValueError("home_advice must be a list")

    data["home_advice"] = [
        key for key in data["home_advice"]
        if key in VALID_ADVICE
    ]

    return data

def classify(summary_text):
    prompt = f"""<start_of_turn>user
You are an expert pediatric triage assistant. 
Analyze the following clinical observations for a child aged 6-12 and classify the triage level.

URGENCY LEVELS:
RED – Emergency: Immediate hospital/ER required. Life-threatening or unstable symptoms.
YELLOW – Urgent: Visit a doctor/clinic within 24 hours. Symptoms are worsening but currently stable.
GREEN – Observation: Home care and monitoring. Minor symptoms.

INSTRUCTIONS:
1. Carefully review the "Clinical Observations" provided.
2. Provide your findings only in a valid JSON format.
3. Select the most relevant advice keys from the ADVICE_LIBRARY below (select 2-3 items).

ADVICE_LIBRARY:
- REST: Ensure the child gets adequate rest.
- FLUIDS: Encourage frequent intake of clean fluids.
- LIGHT_DIET: Provide light, easily digestible food.
- HYGIENE: Maintain proper hand hygiene.
- MONITOR_SYMPTOMS: Monitor symptoms closely for any worsening.
- TEMPERATURE_CHECK: Check temperature periodically if fever is present.

Return format:
{{
  "triage_level": "RED/YELLOW/GREEN",
  "reasoning": "A concise clinical explanation focusing on the severity and combination of symptoms provided.",
  "confidence": "High/Medium/Low",
  "home_advice": ["KEY1", "KEY2"]
}}

Clinical Observations:
{summary_text}<end_of_turn>
<start_of_turn>model
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=400,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_tokens = output[0][inputs["input_ids"].shape[-1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # Try robust JSON extraction
    try:
        res = extract_json_response(response)
        # Confidence auto-bump for RED
        if res.get("triage_level") == "RED":
            res["confidence"] = "High (Model + Structured Assessment)"
        return res
    except Exception as e:
        # Fallback if AI fails JSON        
        return {
    "triage_level": "YELLOW",
    "reasoning": "AI could not confidently analyze the case. As a precaution, medical evaluation is recommended.",
    "confidence": "Low",
    "home_advice": []
        }

# -----------------------------
# LANGUAGE OUTPUT FORMATTERS
# -----------------------------
def malayalam_output(triage_level):
    if triage_level == "RED":
        return """🔴 **അടിയന്തര ചികിത്സ ആവശ്യമാണ് (Emergency)**
ഉടൻ ഒരു പീഡിയാട്രിക് അത്യാഹിത വിഭാഗത്തിൽ (ER) എത്തിക്കുക.
⚠️ ശ്വാസം മുട്ടൽ, ബോധക്ഷയം, കഠിനമായ രക്തസ്രാവം എന്നിവ ഉണ്ടെങ്കിൽ ഒട്ടും വൈകിക്കരുത്."""
    elif triage_level == "YELLOW":
        return """🟡 **ഡോക്ടറെ കാണുക (Urgent Care)**
അടുത്ത 24 മണിക്കൂറിനുള്ളിൽ നിങ്ങളുടെ ഡോക്ടറെയോ അടുത്തുള്ള ക്ലിനിക്കോ സന്ദർശിക്കുക.
⚠️ ലക്ഷണങ്ങൾ കൂടുകയാണെങ്കിൽ ഉടൻ ആശുപത്രിയിൽ പോകുക."""
    elif triage_level == "GREEN":
        return """🟢 **വീട്ടിൽ വിശ്രമിക്കാം (Home Care)**
നിലവിൽ ലക്ഷണങ്ങൾ ഗുരുതരമല്ല. താഴെ നൽകിയിരിക്കുന്ന നിർദ്ദേശങ്ങൾ പാലിക്കുക.
⚠️ പനി കൂടുകയോ കുട്ടി അവശനാകുകയോ ചെയ്താൽ ഡോക്ടറെ സമീപിക്കുക."""
    else:
        return """⚠️ **സാങ്കേതിക തകരാർ (System Error)**
നിങ്ങളുടെ വിവരങ്ങൾ വിശകലനം ചെയ്യാൻ ഇപ്പോൾ സാധ്യമാകുന്നില്ല. കുട്ടിക്ക് അസ്വാസ്ഥ്യമുണ്ടെങ്കിൽ ദയവായി ഒരു ഡോക്ടറെ സമീപിക്കുക."""

def english_output(triage_level):
    if triage_level == "RED":
        return """🔴 **Emergency Care Required**
Take the child to the Pediatric Emergency Room (ER) immediately.
⚠️ Do not delay if there is breathing difficulty, loss of consciousness, or heavy bleeding."""
    elif triage_level == "YELLOW":
        return """🟡 **Urgent Care**
Visit your pediatrician or an urgent care clinic within the next 24 hours.
⚠️ seek immediate care if symptoms worsen."""
    elif triage_level == "GREEN":
        return """🟢 **Home Care & Observation**
Symptoms are currently stable. Follow the home care instructions below.
⚠️ Contact a doctor if fever increases or the child becomes very weak."""
    else:
        return """⚠️ **System Error**
We are unable to process the assessment at this time. Please consult a doctor if you are concerned."""

# -----------------------------
# TERMINAL UI HELPERS
# -----------------------------
def get_terminal_input(prompt, input_type="text", options=None, format_func=None):
    while True:
        if options:
            print(f"\n{prompt}")
            for i, opt in enumerate(options):
                label = format_func(opt) if format_func else opt
                print(f"  {i+1}. {label}")
            val = input("Select option (number): ").strip()
            if val.isdigit() and 1 <= int(val) <= len(options):
                return options[int(val)-1]
            print("Invalid selection. Please try again.")
        else:
            val = input(f"\n{prompt}: ").strip()
            if input_type == "number":
                if val.isdigit():
                    return int(val)
                print("Invalid number. Please try again.")
            else:
                return val

def main():
    print("\n" + "="*50)
    print("🏥 Pediatric Triage System (Age 6–12)")
    print("="*50)

    # 1. Language Selection
    lang_choice = get_terminal_input(
        "Select Language / ഭാഷ തിരഞ്ഞെടുക്കുക:",
        options=["English", "Malayalam"]
    )
    l_key = "en" if lang_choice == "English" else "ml"
    
    answers = {}

    # 2. General Questions
    print("\n--- General Questions / പൊതുവായ ചോദ്യങ്ങൾ ---")
    q1 = QUESTIONS["General"]["Q1"]
    while True:
        age = get_terminal_input(q1[l_key], input_type="number")
        if q1["min"] <= age <= q1["max"]:
            answers["Q1"] = age
            break
        print(f"Age must be between {q1['min']} and {q1['max']}.")

    for q_id in ["Q2", "Q3", "Q4", "Q5"]:
        q_data = QUESTIONS["General"][q_id]
        opts = list(q_data["options"].keys())
        answers[q_id] = get_terminal_input(
            q_data[l_key],
            options=opts,
            format_func=lambda x: q_data["options"][x][l_key]
        )
        if q_id == "Q3" and answers[q_id] == "Yes":
            print("⚠️ WARNING: High risk flag.")

    # 3. Category Selection
    categories = ["Cold / Cough / Fever", "Stomach Pain / Diarrhea / Vomiting", "Breathing Problem", "Body Pain / Headache"]
    print("\n--- Symptom Categories / ലക്ഷണങ്ങൾ ---")
    print("Choose symptoms (comma separated numbers, e.g., 1,3 or press Enter if none):")
    for i, cat in enumerate(categories):
        print(f"  {i+1}. {cat}")
    
    cat_input = input("Selection: ").strip()
    selected_cats = []
    if cat_input:
        indices = [i.strip() for i in cat_input.split(",")]
        for idx in indices:
            if idx.isdigit() and 1 <= int(idx) <= len(categories):
                selected_cats.append(categories[int(idx)-1])

    # 4. Dynamic Category Questions
    for cat in selected_cats:
        print(f"\n--- {cat} ---")
        for q_id, q_data in QUESTIONS[cat].items():
            opts = list(q_data["options"].keys())
            answers[q_id] = get_terminal_input(
                q_data[l_key],
                options=opts,
                format_func=lambda x: q_data["options"][x][l_key]
            )
            if q_data.get("is_critical") and answers[q_id] == "Yes":
                print("⚠️ WARNING: High risk flag.")

    # 5. Critical Questions
    print("\n--- Critical Conditions / ഗുരുതരമായ അവസ്ഥകൾ ---")
    for q_id, q_data in QUESTIONS["Critical Red-Flags"].items():
        opts = list(q_data["options"].keys())
        answers[q_id] = get_terminal_input(
            q_data[l_key],
            options=opts,
            format_func=lambda x: q_data["options"][x][l_key]
        )
        if answers[q_id] == "Yes":
            print("⚠️ CRITICAL: RED Flag / ഗുരുതരമായ ലക്ഷണമാണ്.")

    # 6. Result Calculation
    print("\n" + "="*50)
    print("Analyzing Urgency...")
    
    # Rule-based check
    red_flag_data = check_red_flags(answers)
    
    if red_flag_data:
        res_json = red_flag_data
    else:
        # AI-based check
        summary = build_summary(answers)
        res_json = classify(summary)
    
    # Display Triage Advice
    if l_key == "ml":
        advice_msg = malayalam_output(res_json["triage_level"])
    else:
        advice_msg = english_output(res_json["triage_level"])
    
    print("\n" + advice_msg)
    
    # Show Home Care Advice
    if res_json.get("home_advice") and res_json["triage_level"] != "RED":
        print("\n--- Home Care Advice / വീട്ടുചികിത്സാ നിർദ്ദേശങ്ങൾ ---")
        for key in res_json["home_advice"]:
            advice_item = HOME_ADVICE_LIBRARY.get(key)
            if advice_item:
                print(f"- {advice_item[l_key]}")
    
    # Show Reasoning
    print("\n" + "-"*30)
    print(f"Reasoning: {res_json['reasoning']}")
    print(f"Confidence: {res_json['confidence']}")
    print("-"*30)
    
    print("\nDisclaimer: This tool is for informational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment.")
    print("="*50 + "\n")

if __name__ == "__main__":
    main()



🏥 Pediatric Triage System (Age 6–12)

Select Language / ഭാഷ തിരഞ്ഞെടുക്കുക:
  1. English
  2. Malayalam


Select option (number):  1



--- General Questions / പൊതുവായ ചോദ്യങ്ങൾ ---



What is the child’s age?:  12



How long has the problem been present?
  1. < 1 day
  2. 1–2 days
  3. 3+ days


...

Select option (number):  2



Is there a severe injury or heavy bleeding?
  1. Yes
  2. No


Select option (number):  2


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Analyzing Urgency...

🟡 **Urgent Care**
Visit your pediatrician or an urgent care clinic within the next 24 hours.
⚠️ seek immediate care if symptoms worsen.

------------------------------
Reasoning: AI could not confidently analyze the case. As a precaution, medical evaluation is recommended.
Confidence: Low
------------------------------

Disclaimer: This tool is for informational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment.

